# Using ObjectScript in a notebook

In [1]:
Set hello = "helloworld2"
zw hello

hello="helloworld2"


# Using embedded Python from ObjectSCript

In [2]:
set datetime = ##class(%SYS.Python).Import("datetime")
zw datetime
zw datetime.date.today().isoformat()

datetime=3@%SYS.Python  ; <module 'datetime' from '/usr/lib/python3.8/datetime.py'>  ; <OREF>
"2022-03-28"


In [3]:
set rslt = ##class(%SYS.Python).Run("print('hello world')")

hello world


In [7]:
zn "USER"

## Use reportlab python library to generate a PDF from iris

In [20]:
do ##class(Demo.reportlab).CreateSamplePDF("/opt/irisapp/misc/sample.pdf")